In [141]:
import pandas as pd
import numpy as np
import pickle

In [9]:
import datetime

In [10]:
from models.hmm_seq_fit import Scoring

In [11]:
df = pd.read_csv('/home/kasper/Dropbox/Scrapping/soccerway/csv/final_data_soccerway.csv')
df.loc[:,'date'] = pd.to_datetime(df.date)

input_date = datetime.datetime(2018,1,31)

input_data = df[df.date==input_date]

home_teams = input_data.home_team.dropna().drop_duplicates().values
teams_grouped = df.groupby('home_team').size()
home_teams_consider = teams_grouped[home_teams].sort_values(ascending=False)

away_teams = input_data.away_team.dropna().drop_duplicates().values
teams_grouped = df.groupby('away_team').size()
away_teams_consider = teams_grouped[away_teams].sort_values(ascending=False)

threshold = 100
home_final_teams = home_teams_consider[home_teams_consider > threshold].index
away_final_teams = away_teams_consider[away_teams_consider > threshold].index

final_matches = input_data[(input_data.home_team.isin(home_final_teams)) & (input_data.away_team.isin(away_final_teams))]

# START FUNCTIONS

In [142]:
def get_date_x_y_for_team_alone(df,side,result_key,team):
    if side is None:
        da = df[(df['home_team']==team)|(df['away_team']==team)]
        ix = da[result_key].dropna().index
        dates = df.loc[ix,'date']
        ys = scores.loc[ix].values.ravel()
        xs = da.loc[ix,result_key].values
        return dates, xs, ys, ix
    else:
        da = df[(df[side]==team)]
        ix = da[result_key].dropna().index
        dates = df.loc[ix,'date']
        ys = scores.loc[ix].values.ravel()
        xs = da.loc[ix,result_key].values
        return dates, xs, ys, ix

In [143]:
def get_date_x_y_for_match(df,result_key, teamA, teamB):
    da = df[(df['home_team']==teamA) & (df['away_team']==teamB)]
    ix = da[result_key].dropna().index
    dates = df.loc[ix,'date']
    ysA = scores.loc[ix,teamA].values.ravel()
    ysB = scores.loc[ix,teamB].values.ravel()
    xs = da.loc[ix,result_key].values
    return dates, xs, ysA, ysB, ix

# END FUNCTIONS

## Get Elo-Ratings

In [ ]:
! sshpass -p slave0123 scp -r slave0@192.168.0.106:~/elo.csv /home/kasper/PycharmProjects/PGMS/data/elo_ratings/ratings/elo.csv

# Get Xs and Ys sequences

In [144]:
scores = pd.read_csv('./data/elo_ratings/ratings/elo.csv', index_col='Unnamed: 0')
scores.head(1)

,APOEL,Apollon
149,100.0,100.0


In [146]:
matches = [['APOEL','Apollon']] # get from final_data?
result_key = 'over_under_0.5'

In [147]:
jfile = {}

sides = ['home_team', 'away_team', None]

for match in matches:
    print(match)
    for team in match:
        print(team)
        outputs = []
        for side in sides:
            d,x,y,ix = get_date_x_y_for_team_alone(df, result_key=result_key, team=team, side=side)
            output = pd.DataFrame(list(zip(d,x,y)),index = ix, columns=['date','xs','ys'])
            outputs.append(output)
        temp = {result_key:outputs}
        jfile[team] = temp

['APOEL', 'Apollon']
APOEL
Apollon


In [149]:
m_outputs = []
for match in matches:
    print(match)
    d,x,yA,yB,ix = get_date_x_y_for_match(df,result_key=result_key,teamA=match[0],teamB=match[1])
    output = pd.DataFrame(list(zip(d,x,yA,yB)),index = ix, columns=['date', 'xs', 'yA', 'yB'])
    m_outputs.append(output)
    jfile[match[0]+':'+match[1]] = {result_key:outputs}

['APOEL', 'Apollon']


In [151]:
with open('data.pkl', 'wb') as fp:
    pickle.dump(jfile, fp)